# <span style="font-width:bold; font-size: 3rem; color:#2656a3;">**Data Engineering and Machine Learning Operations in Business** </span> <span style="font-width:bold; font-size: 3rem; color:#333;">- Part 01: Feature Backfill</span>

## 🗒️ The notebook is divided into the following sections:
1. Loading data and process features
2. Connecting to Hopsworks Feature Store
3. Creating feature groups and uploading them to the feature store

## <span style='color:#2656a3'> ⚙️ Import of Libraries and Packages

First, we install the Python packages required for this notebook. We'll use the --quiet command after specifying the names of the libraries to ensure a silent installation process. Then, we proceed to import all the necessary libraries.

In [25]:
# Install of the packages for hopsworks
# !pip install -U hopsworks --quiet

In [26]:
# First we go one back in our directory to access the folder with our functions
%cd ..

# Now we import the functions from the features folder
from features import electricity_prices, weather_measures # Own functions. This is functions we have created 

# We go back into the notebooks folder
%cd notebooks

/Users/camillahannesbo/Documents/AAU/Master - BDS/2. semester/Data Engineering and Machine learning operations in Business/MLOPs-Assignment-
/Users/camillahannesbo/Documents/AAU/Master - BDS/2. semester/Data Engineering and Machine learning operations in Business/MLOPs-Assignment-/notebooks


In [27]:
# Importing the packages for the needed libraries for the Jupyter notebook
import pandas as pd
import requests
from datetime import datetime, timedelta

# Ignore warnings
import warnings 
warnings.filterwarnings('ignore')

## <span style="color:#2656a3;"> 💽 Load the Historical Data

The data used comes from different sources:

- Electricity prices in Denmark on hourly basis per day from [Energinet](https://www.energidataservice.dk). Loacated in the *featuresfolder* under electricity_prices.
- Different meteorological observations based on Aalborg Denmark from [Open Meteo](https://www.open-meteo.com). Loacated in the *featuresfolder* under weather_measures.
- Danish calendar that categorizes dates into types based on whether it is a weekday or not. This files is made manually by the group and is located in the *datafolder* inside this repository.
- Forecast Renewable Energy next day from [Energinet](https://www.energidataservice.dk). Loacated in the *featuresfolder* under electricity_prices.
- Weather Forecast based on Aalborg Denmark from [Open Meteo](https://www.open-meteo.com). Loacated in the *featuresfolder* under weather_measures. (This data is used later to parse in new real-time weather data)


### <span style="color:#2656a3;">💸 Electricity prices per day from Energinet
This first dataset is Electricity prices on hourly basis per day from Energinet/Dataservice.

In [64]:
# Fetching historical electricity prices for area DK1 from January 1, 2022
# Note: The end date is currently left out to retrieve data up to the present date of yesterday 
# Today is not included in the data as it is not historical data
electricity_df = electricity_prices.electricity_prices(
    historical=True, 
    area=["DK1"], 
    start='2022-01-01', 
    #end='2023-12-31'
)

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


In [29]:
# Display the first 5 rows of the electricity dataframe
electricity_df.head(5)

,timestamp,time,date,dk1_spotpricedkk_kwh
0,1640995200000,2022-01-01 00:00:00,2022-01-01,0.37220
1,1640998800000,2022-01-01 01:00:00,2022-01-01,0.30735
2,1641002400000,2022-01-01 02:00:00,2022-01-01,0.32141
3,1641006000000,2022-01-01 03:00:00,2022-01-01,0.33806
4,1641009600000,2022-01-01 04:00:00,2022-01-01,0.28013


In [30]:
# Display the last 5 rows of the electricity dataframe
electricity_df.tail(5)

,timestamp,time,date,dk1_spotpricedkk_kwh
20296,1714071600000,2024-04-25 19:00:00,2024-04-25,1.05217
20297,1714075200000,2024-04-25 20:00:00,2024-04-25,1.13570
20298,1714078800000,2024-04-25 21:00:00,2024-04-25,0.84799
20299,1714082400000,2024-04-25 22:00:00,2024-04-25,0.68124
20300,1714086000000,2024-04-25 23:00:00,2024-04-25,0.59466


In [31]:
# Showing the information for the electricity dataframe
electricity_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20301 entries, 0 to 20300
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   timestamp             20301 non-null  int64         
 1   time                  20301 non-null  datetime64[ns]
 2   date                  20301 non-null  object        
 3   dk1_spotpricedkk_kwh  20301 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 634.5+ KB


### <span style="color:#2656a3;">☀️💨 Forecast Renewable Energy next day from Energinet
Second dataset is Forecast Renewable Energy next day also on hourly basis from Energinet

In [32]:
# Fetching historical forecast of renewable energy data for area DK1 from January 1, 2022
# Note: The end date is currently left out to retrieve data up to the present date of yesterday 
# Today is not included in the data as it is not historical data
forecast_renewable_energy_df = electricity_prices.forecast_renewable_energy(
    historical=True, 
    area = ["DK1"],
    start= '2022-01-01', 
    #end='2023-12-31'
)

In [33]:
# Display the first 5 rows of the forecast_renewable_energy dataframe
forecast_renewable_energy_df.head(5)

,timestamp,time,date,dk1_offshore_wind_forecastintraday_kwh,dk1_onshore_wind_forecastintraday_kwh,dk1_solar_forecastintraday_kwh
0,1641024000000,2022-01-01 08:00:00,2022-01-01,0.611708,0.236792,0.000050
1,1641027600000,2022-01-01 09:00:00,2022-01-01,0.459708,0.196667,0.004841
2,1641031200000,2022-01-01 10:00:00,2022-01-01,0.310375,0.178500,0.020353
3,1641034800000,2022-01-01 11:00:00,2022-01-01,0.320750,0.201125,0.035719
4,1641038400000,2022-01-01 12:00:00,2022-01-01,0.355667,0.277667,0.038027


In [34]:
# Display the last 5 rows of the forecast_renewable_energy dataframe
forecast_renewable_energy_df.tail(5)

,timestamp,time,date,dk1_offshore_wind_forecastintraday_kwh,dk1_onshore_wind_forecastintraday_kwh,dk1_solar_forecastintraday_kwh
14282,1714071600000,2024-04-25 19:00:00,2024-04-25,0.062917,0.277500,0.115812
14283,1714075200000,2024-04-25 20:00:00,2024-04-25,0.089208,0.266250,0.013700
14284,1714078800000,2024-04-25 21:00:00,2024-04-25,0.110000,0.299208,0.000467
14285,1714082400000,2024-04-25 22:00:00,2024-04-25,0.131417,0.357833,0.000000
14286,1714086000000,2024-04-25 23:00:00,2024-04-25,0.185875,0.372000,0.000000


In [35]:
# Showing the information for the forecast_renewable_energy dataframe
forecast_renewable_energy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14287 entries, 0 to 14286
Data columns (total 6 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   timestamp                               14287 non-null  int64         
 1   time                                    14287 non-null  datetime64[ns]
 2   date                                    14287 non-null  object        
 3   dk1_offshore_wind_forecastintraday_kwh  14271 non-null  float64       
 4   dk1_onshore_wind_forecastintraday_kwh   14271 non-null  float64       
 5   dk1_solar_forecastintraday_kwh          14271 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 669.8+ KB


### <span style="color:#2656a3;"> 🌤 Weather measurements from Open Meteo
Next weather measurements from Open Meteo is fetched.

#### <span style="color:#2656a3;"> 🕰️ Historical Weather Measures

In [36]:
# Fetching historical weather measurements from January 1, 2022
# Note: The end date is currently left out to retrieve data up to the present date of yesterday 
# Today is not included in the data as it is not historical data
historical_weather_df = weather_measures.historical_weather_measures(
    historical=True, 
    start = '2022-01-01', 
    #end = '2023-12-31'
)

In [37]:
# Display the first 5 rows of the weather dataframe
historical_weather_df.head(5)

,timestamp,date,time,temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m
0,1640995200000,2022-01-01,2022-01-01 00:00:00,6.7,100.0,0.0,0.0,0.0,3.0,100.0,16.2,36.0
1,1640998800000,2022-01-01,2022-01-01 01:00:00,6.6,100.0,0.0,0.0,0.0,3.0,100.0,16.2,30.2
2,1641002400000,2022-01-01,2022-01-01 02:00:00,6.7,99.0,0.0,0.0,0.0,3.0,100.0,15.5,30.6
3,1641006000000,2022-01-01,2022-01-01 03:00:00,6.7,100.0,0.0,0.0,0.0,3.0,100.0,12.7,28.8
4,1641009600000,2022-01-01,2022-01-01 04:00:00,6.7,99.0,0.0,0.0,0.0,3.0,100.0,10.6,23.8


In [38]:
# Display the last 5 rows of the weather dataframe
historical_weather_df.tail(5)

,timestamp,date,time,temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m
20275,1713985200000,2024-04-24,2024-04-24 19:00:00,3.9,88.0,0.0,0.0,0.0,3.0,87.0,7.1,11.9
20276,1713988800000,2024-04-24,2024-04-24 20:00:00,3.4,91.0,0.0,0.0,0.0,3.0,100.0,6.0,9.4
20277,1713992400000,2024-04-24,2024-04-24 21:00:00,4.2,89.0,0.0,0.0,0.0,3.0,99.0,5.6,10.4
20278,1713996000000,2024-04-24,2024-04-24 22:00:00,4.0,88.0,0.0,0.0,0.0,3.0,100.0,5.4,10.4
20279,1713999600000,2024-04-24,2024-04-24 23:00:00,3.9,88.0,0.0,0.0,0.0,3.0,100.0,7.5,12.6


In [39]:
# Showing the information for the weather dataframe
historical_weather_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20280 entries, 0 to 20279
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   timestamp             20280 non-null  int64         
 1   date                  20280 non-null  object        
 2   time                  20280 non-null  datetime64[ns]
 3   temperature_2m        20280 non-null  float64       
 4   relative_humidity_2m  20280 non-null  float64       
 5   precipitation         20280 non-null  float64       
 6   rain                  20280 non-null  float64       
 7   snowfall              20280 non-null  float64       
 8   weather_code          20280 non-null  float64       
 9   cloud_cover           20280 non-null  float64       
 10  wind_speed_10m        20280 non-null  float64       
 11  wind_gusts_10m        20280 non-null  float64       
dtypes: datetime64[ns](1), float64(9), int64(1), object(1)
memory usage: 2.0+ M

In [40]:
# deleting rows with missing values
# historical_weather_df.dropna()

In [41]:
# historical_weather_df.tail(5)

#### <span style="color:#2656a3;"> 🌈 Weather Forecast
Weather Forecast from Open Meteo is now being fetched. This data is used in part 02 the feature_pipeline to parse in new real-time weather data.

In [42]:
# Fetching weather forecast measures for the next 5 days
weather_forecast_df = weather_measures.forecast_weather_measures(
    forecast_length=5
)

In [43]:
# Display the first 5 rows of the weather_forecast dataframe
weather_forecast_df.head(5)

,timestamp,date,time,temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m
0,1714089600000,2024-04-26,2024-04-26 00:00:00,6.4,85,0.1,0.1,0.0,51,49,10.8,21.6
1,1714093200000,2024-04-26,2024-04-26 01:00:00,5.8,87,0.0,0.0,0.0,2,71,10.8,18.4
2,1714096800000,2024-04-26,2024-04-26 02:00:00,5.4,90,0.0,0.0,0.0,3,100,14.8,25.6
3,1714100400000,2024-04-26,2024-04-26 03:00:00,4.9,92,0.0,0.0,0.0,2,74,11.5,19.8
4,1714104000000,2024-04-26,2024-04-26 04:00:00,4.6,92,0.0,0.0,0.0,2,55,13.3,23.8


In [44]:
# Display the last 5 rows of the weather_forecast dataframe
weather_forecast_df.tail(5)

,timestamp,date,time,temperature_2m,relative_humidity_2m,precipitation,rain,snowfall,weather_code,cloud_cover,wind_speed_10m,wind_gusts_10m
115,1714503600000,2024-04-30,2024-04-30 19:00:00,12.4,69,0.0,0.0,0.0,1,67,10.3,24.5
116,1714507200000,2024-04-30,2024-04-30 20:00:00,11.4,74,0.0,0.0,0.0,1,45,9.5,22.0
117,1714510800000,2024-04-30,2024-04-30 21:00:00,10.7,79,0.0,0.0,0.0,1,24,9.2,19.8
118,1714514400000,2024-04-30,2024-04-30 22:00:00,10.4,83,0.0,0.0,0.0,1,23,9.6,20.5
119,1714518000000,2024-04-30,2024-04-30 23:00:00,10.4,87,0.0,0.0,0.0,1,23,10.4,21.2


In [45]:
# Showing the information for the weather weather_forecast dataframe
weather_forecast_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   timestamp             120 non-null    int64         
 1   date                  120 non-null    object        
 2   time                  120 non-null    datetime64[ns]
 3   temperature_2m        120 non-null    float64       
 4   relative_humidity_2m  120 non-null    int64         
 5   precipitation         120 non-null    float64       
 6   rain                  120 non-null    float64       
 7   snowfall              120 non-null    float64       
 8   weather_code          120 non-null    int64         
 9   cloud_cover           120 non-null    int64         
 10  wind_speed_10m        120 non-null    float64       
 11  wind_gusts_10m        120 non-null    float64       
dtypes: datetime64[ns](1), float64(6), int64(4), object(1)
memory usage: 11.4+ KB


### <span style="color:#2656a3;"> 🗓️ Calendar of Danish workdays and holidays 
Lastly the calender data is being loaded in.

In [46]:
# Read csv file with calender
calender_df = pd.read_csv('https://raw.githubusercontent.com/Camillahannesbo/MLOPs-Assignment-/main/data/calendar_incl_holiday.csv', delimiter=';', usecols=['date', 'type'])
 
# Display the DataFrame
calender_df.head()

,date,type
0,01/01/2022,Not a Workday
1,02/01/2022,Not a Workday
2,03/01/2022,Workday
3,04/01/2022,Workday
4,05/01/2022,Workday


In [47]:
# Formatting the date column to 'YYYY-MM-DD' dateformat
calender_df["date"] = calender_df["date"].map(lambda x: datetime.strptime(x, '%d/%m/%Y').strftime("%Y-%m-%d"))

In [48]:
# Display the first 5 rows of the calender dataframe
calender_df.head(5)

,date,type
0,2022-01-01,Not a Workday
1,2022-01-02,Not a Workday
2,2022-01-03,Workday
3,2022-01-04,Workday
4,2022-01-05,Workday


In [49]:
# Display the last 5 rows of the calender dataframe
calender_df.tail(5)

,date,type
1091,2024-12-27,Workday
1092,2024-12-28,Not a Workday
1093,2024-12-29,Not a Workday
1094,2024-12-30,Workday
1095,2024-12-31,Workday


In [50]:
# Showing the information for the calender dataframe
calender_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1096 entries, 0 to 1095
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    1096 non-null   object
 1   type    1096 non-null   object
dtypes: object(2)
memory usage: 17.3+ KB


## <span style="color:#2656a3;"> 📡 Connecting to Hopsworks Feature Store

We connect to Hopsworks Feature Store so we can access and create feature groups.

In [51]:
# Importing the hopsworks module
import hopsworks

# Logging in to the Hopsworks project
project = hopsworks.login()

# Getting the feature store from the project
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/550040
Connected. Call `.close()` to terminate connection gracefully.


### <span style="color:#2656a3;"> 🪄 Creating Feature Groups
A feature group can be seen as a collection of conceptually related features. In this case we create feature groups for the 
- eletricity price data,
- weather data,
- calender data, and
- forecast_renewable_energy.

We specify a `primary_key` as `date`, so we are able to join them when we create a dataset for training later in part 03 the training_pipeline.
We define a name and a short describtion of the feature group's contents and a version number. 

`event_time` is specifyed as `timestamp`.

We've set `online_enabled` to `True` to enable accessing the feature group through the Online API for a Feature View.

In [52]:
# Creating the feature group for the weather data
weather_fg = fs.get_or_create_feature_group(
    name="weather_measurements",
    version=1,
    description="Weather measurements from Open Meteo API",
    primary_key=["date"],
    event_time="timestamp",
    online_enabled=True,
)

We have now outlined metadata for the feature group. Data hasn't been stored yet, and there's no schema defined. To store data persistently for the feature group, we populate it with its associated data using the `insert` function.

In [53]:
# Inserting the weather_df into the feature group named weather_fg
weather_fg.insert(historical_weather_df)

Uploading Dataframe: 100.00% |██████████| Rows 20280/20280 | Elapsed Time: 00:08 | Remaining Time: 00:00


Launching job: weather_measurements_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/550040/jobs/named/weather_measurements_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x129748450>, None)

We make a descriptions for each feature we put into the feature group. In this way we are adding more information and documentation to the user.

In [54]:
# List of descriptions for weather features
weather_feature_descriptions = [
    {"name": "timestamp", "description": "Timestamp for the event_time"},
    {"name": "date", "description": "Date of the weather measurement"},
    {"name": "time", "description": "Time of the weather measurement"},
    {"name": "temperature_2m", "description": "Temperature at 2m above ground"},
    {"name": "relative_humidity_2m", "description": "Relative humidity at 2m above ground"},
    {"name": "precipitation", "description": "Precipitation"},
    {"name": "rain", "description": "Rain"},
    {"name": "snowfall", "description": "Snowfall"},   
    {"name": "weather_code", "description": "Weather code"},   
    {"name": "cloud_cover", "description": "Cloud cover"},   
    {"name": "wind_speed_10m", "description": "Wind speed at 10m above ground"},   
    {"name": "wind_gusts_10m", "description": "Wind gusts at 10m above ground"},   
]

# Updating feature descriptions
for desc in weather_feature_descriptions: 
    weather_fg.update_feature_description(desc["name"], desc["description"])

We replicate the process for both the `electricity_fg`, `forecast_renewable_energy_fg` and `danish_holidays_fg` by establishing feature groups and inserting the dataframes into their respective feature groups.

In [55]:
# Creating the feature group for the electricity prices
electricity_fg = fs.get_or_create_feature_group(
    name="electricity_prices",
    version=1,
    description="Electricity prices from Energidata API",
    primary_key=["date"],
    online_enabled=True,
    event_time="timestamp",
)

In [56]:
# Inserting the electricity_df into the feature group named electricity_fg
electricity_fg.insert(electricity_df)

Uploading Dataframe: 100.00% |██████████| Rows 20301/20301 | Elapsed Time: 00:07 | Remaining Time: 00:00


Launching job: electricity_prices_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/550040/jobs/named/electricity_prices_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1296159d0>, None)

In [57]:
# List of descriptions for electricity features
electricity_feature_descriptions = [
    {"name": "timestamp", "description": "Timestamp for the event_time"},
    {"name": "date", "description": "Date of the electricity measurement"},
    {"name": "time", "description": "Time of the electricity measurement"},
    {"name": "dk1_spotpricedkk_kwh", "description": "Spot price in DKK per KWH"}, 
]

# Updating feature descriptions
for desc in electricity_feature_descriptions: 
    electricity_fg.update_feature_description(desc["name"], desc["description"])

In [58]:
# Creating the feature group for the electricity prices
forecast_renewable_energy_fg = fs.get_or_create_feature_group(
    name="forecast_renewable_energy",
    version=1,
    description="Forecast on Renewable Energy on ForecastType from Energidata API",
    primary_key=["date"],
    online_enabled=True,
    event_time="timestamp",
)

In [59]:
# Inserting the electricity_df into the feature group named electricity_fg
forecast_renewable_energy_fg.insert(forecast_renewable_energy_df)

Uploading Dataframe: 100.00% |██████████| Rows 14287/14287 | Elapsed Time: 00:07 | Remaining Time: 00:00


Launching job: forecast_renewable_energy_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/550040/jobs/named/forecast_renewable_energy_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x129227610>, None)

In [60]:
# List of descriptions for forecast_renewable_energy features
forecast_renewable_energy_feature_descriptions = [
    {"name": "timestamp", "description": "Timestamp for the event_time"},
    {"name": "date", "description": "Date"},
    {"name": "time", "description": "Time for the event_time"},
    {"name": "dk1_offshore_wind_forecastintraday_kwh", "description": "The forecast for the coming day at 6am Danish time zone"},
]

# Updating feature descriptions
for desc in forecast_renewable_energy_feature_descriptions: 
    forecast_renewable_energy_fg.update_feature_description(desc["name"], desc["description"])

In [61]:
# Creating the feature group for the danish holidays
danish_holidays_fg = fs.get_or_create_feature_group(
    name="danish_holidays",
    version=1,
    description="Danish holidays calendar.",
    online_enabled=True,
    primary_key=["date"],
)

In [62]:
# Inserting the calender_df into the feature group named danish_holidays_fg
danish_holidays_fg.insert(calender_df)

Uploading Dataframe: 100.00% |██████████| Rows 1096/1096 | Elapsed Time: 00:06 | Remaining Time: 00:00


Launching job: danish_holidays_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/550040/jobs/named/danish_holidays_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1295fd690>, None)

In [63]:
# List of descriptions for danish_holidays features
danish_holidays_feature_descriptions = [
    {"name": "date", "description": "Date in the calendar"},
    {"name": "type", "description": "Holyday or not holyday"},
]

# Updating feature descriptions
for desc in danish_holidays_feature_descriptions: 
    danish_holidays_fg.update_feature_description(desc["name"], desc["description"])

---
## <span style="color:#2656a3;">⏭️ **Next:** Part 02: Feature Pipeline </span>

Next we will generate new data for the Feature Groups.